## CUDA Best Practices

Lecture derived from https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html.


### Memory Access Patterns

CUDA memory is _banked_ and _direct mapped_.
  * ba
  
#### Coalesced Memory Accesses
<img src="
https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/graphics/coalesced-access.png